# TOPIC2VEC algorithm by using gensim and according to the second hint given by Gordon Mohr.  
(https://groups.google.com/forum/#!topic/gensim/BVu5-pD6910)


1. Vectorization of docs by using CountVectorizer (with or without tfidf) with no lemmatization
2. Latent Dirichlet Allocation 
3. Topic2Vec of the entire dataset (20 NewsGroups)   

It saves:
* the topic2vec model obtained

Imports

In [1]:
import numpy as np; import pandas as pd; import matplotlib.pyplot as plt
%matplotlib inline
import codecs 
from glob import glob
import os
import pickle
import copy
import pyorient
import ast

In [1]:
from __future__ import print_function
from time import time
import string
import re
# random
from random import shuffle

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.decomposition import LatentDirichletAllocation

In [3]:
from gensim import corpora, models, similarities

In [4]:
n_top_words = 20

## 1. IMPORTING DOCS FROM 20 NEWSGROUPS DATASET

In [5]:
from sklearn.datasets import fetch_20newsgroups
categories = ['comp.sys.ibm.pc.hardware',
'comp.sys.mac.hardware',
'comp.windows.x',
'rec.sport.baseball',
'rec.sport.hockey',
'sci.med',
'sci.space',
'soc.religion.christian']

n_topics = len(categories)

categories_source = {}

for cat in categories:
    categories_source[cat] = cat.replace('.', '_')

In [6]:
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)

In [7]:
for i,j in categories_source.items():
    print(i,j)

comp.sys.mac.hardware comp_sys_mac_hardware
comp.sys.ibm.pc.hardware comp_sys_ibm_pc_hardware


#### TOTAL NUMBER OF DOC

In [8]:
n_docs = newsgroups_train.filenames.shape[0]
n_docs

1168

# 2. LDA to find the topic most-associated with each word

## 2.1 From Strings to Vectors

### WITH Lemmatization

### WITHOUT Lemmatization

In [9]:
t0 = time()
tf_vectorizer = CountVectorizer(encoding='utf-8', analyzer='word', stop_words='english',
                                ngram_range = (1,1), min_df = 2, token_pattern = '[a-zA-Z]{2,}').fit(newsgroups_train.data)
print("fit vectorizer without lemmatization done in %0.3fs." % (time() - t0))

fit vectorizer without lemmatization done in 0.125s.


### Vectorization

In [10]:
n_features = len(tf_vectorizer.get_feature_names())

In [11]:
newsgroups_train.data[0]

u"From article <1qq7i1INNdqc@dns1.NMSU.Edu>, by bgrubb@dante.nmsu.edu (GRUBB):\n\n[Tons of stuff deleted on SCSI vs. IDE question...]\n\nWow, you guys are really going wild on this IDE vs. SCSI thing, and I think\nit's great!\n\nLike lots of people, I'd really like to increase my data transfer rate from\nthe hard drive.  Right now I have a 15ms 210Mb IDE drive (Seagate 1239A), and\nwhat I would say is a standard (not special, no cache I believe) IDE controller\ncard on my ISA 486-50.\n\nI'm currently thinking about adding another HD, in the 300Mb to 500Mb range.\nAnd I'm thinking hard (you should hear those gears a-grinding in my head)\nabout buying a SCSI drive (SCSI for the future benefit).  I believe I'm getting\nsomething like 890Kb/sec transfer right now (according to NU).  How would this\nnumber compare if I bought the state-of-the-art SCSI card for my ISA PC, and\nthe state-of-the-art SCSI hard drive (the wailing-est system I could hope for)?\nObviously money factors into this c

In [12]:
tf_docs = tf_vectorizer.transform(newsgroups_train.data)

### WITH TFIDF

tfidf_vectorizer = TfidfTransformer(sublinear_tf=False, use_idf = True).fit(tf_docs)
tfidf_docs = tfidf_vectorizer.transform(tf_docs)

## 2.2 LDA implementation

In [13]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [15]:
print("Fitting LDA models with tf features, n_samples=%d and n_features=%d..."
      % (n_docs, n_features))
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,
                                learning_method='online', learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf_docs)
print("done in %0.3fs." % (time() - t0))

Fitting LDA models with tf features, n_samples=1168 and n_features=4244...
done in 1.569s.


In [16]:
print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


Topics in LDA model:
Topic #0:
drive disk edu drives bios hard controller st floppy com os jumper master slave rom pin ide card mb interface
Topic #1:
scsi mb mac card use does know drive like apple just problem bit thanks don need new pc bus work



In [17]:
per_topic_distr_LDA = lda.components_
per_topic_distr_LDA.shape
#per_topic_distr_LDA.sum(axis=1)

(2, 4244)

# 3. TOPIC2VEC

In [18]:
most_p_topic = np.argmax(per_topic_distr_LDA, axis=0)

In [19]:
word_and_topic = zip(tf_feature_names, most_p_topic)

word2topic_dict = {word : 'topic_' + np.array_str(topic) for word, topic in word_and_topic}

## 3.1 Tokenization

In [20]:
def tokenizer(document):
    
    text = "".join([ch for ch in document if ch not in string.punctuation])
    text_list = text.split()
    normalized_text = [x.lower() for x in text_list]
    # Define an empty list
    nostopwords_text = []
    # Scan the words
    for word in normalized_text:
        # Determine if the word is contained in the stop words list
        if word not in ENGLISH_STOP_WORDS:
            # If the word is not contained I append it
            nostopwords_text.append(word)
    tokenized_text = [word for word in nostopwords_text if re.search('[a-zA-Z]{2,}', word)]
            
    return tokenized_text

In [21]:
def map_doc_to_topic(tokenized_text, prefix, doc_id_number, word2topic_dict):
    doc_to_topic_list = [prefix + '_' + str(doc_id_number)]
    for word in tokenized_text:
        if word in word2topic_dict.keys():
            doc_to_topic_list.append(word2topic_dict[word])
            
    return doc_to_topic_list

In [3]:
class LabeledLineSentence_training(object):
    def __init__(self, sources, word2topic_dict):
        self.labels_list = word2topic_dict
        self.sources = sources
        flipped = {}
        # make sure that keys are unique
        for key, value in sources.items():
            if value not in flipped:
                flipped[value] = [key]
            else:
                raise Exception('Non-unique prefix encountered')
    
    def __iter__(self):
        for source, prefix in self.sources.items():
            print(source)
            newsgroups_train_cat = fetch_20newsgroups(subset='train',
                                                      remove=('headers', 'footers', 'quotes'),
                                                      categories=[source])
            for idx, doc in enumerate(newsgroups_train_cat.data):
                words_doc=tokenizer(doc)
                tags_doc = map_doc_to_topic(words_doc, prefix, idx, word2topic_dict)
                yield models.doc2vec.LabeledSentence(words = words_doc,
                                                     tags = tags_doc)
                
    def to_array(self):
        self.sentences = []
        for source, prefix in self.sources.items():
            print(source)
            newsgroups_train_cat = fetch_20newsgroups(subset='train',
                                                      remove=('headers', 'footers', 'quotes'),
                                                      categories=[source])
            for idx, doc in enumerate(newsgroups_train_cat.data):
                words_doc=tokenizer(doc)
                tags_doc = map_doc_to_topic(words_doc, prefix, idx, word2topic_dict)
                self.sentences.append(models.doc2vec.LabeledSentence(words = words_doc,
                                                     tags = tags_doc))
        return self.sentences
            
    def sentences_perm(self):
        shuffle(self.sentences)
        return self.sentences

## 3.1 Training

In [5]:
it = LabeledLineSentence_training(categories_source, word2topic_dict)

NameError: name 'categories_source' is not defined

In [ ]:
model = models.Doc2Vec(size=100, window=10, min_count=4, dm=1, dbow_words=1,
                              workers=50, alpha=0.025, min_alpha=0.025) # use fixed learning rate
model.build_vocab(it.to_array())

In [ ]:
for epoch in range(10):
    model.train(it.sentences_perm())
    model.alpha -= 0.002 # decrease the learning rate
    model.min_alpha = model.alpha # fix the learning rate, no decay

In [ ]:
fname =  os.getcwd() # Prints the working directory
fname = fname + '/topic2vec_20NG_2_ndoc' + str(n_docs) + 'n_topic' + str(n_topics) + '.model'
model.save(fname)